### STEP: 1

Import necessary libraries for **Predicting Future Stock Prices**

In [5]:
# Import required libraries
import numpy as np
import pandas as pd
import hvplot.pandas
import requests

import warnings
warnings.filterwarnings('ignore')

# Libraries to Extract Stock Price Historical Data 
# from Yahoo! Finance using Tickers from Wikipedia
import yfinance as yf
from pandas.tseries.offsets import DateOffset
from datetime import datetime, timedelta
from pandas.tseries.holiday import USFederalHolidayCalendar

# Library for Technical Analysis
import pandas_ta as ta

# Libraries for ML Analysis using Linear Models
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import MultiTaskElasticNet
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge

scaler = StandardScaler()
today = datetime.today().strftime('%Y-%m-%d')

### STEP: 2

Defining various functions used in the code for 
* extracting data, 
* adding technical indicators as features,
* predicting **Close** price and **VWAP** volume-weighted average price,
* using linear models for predicting with best model output.

Best model is selected based on minimum MAPE & RMSE

In [6]:
# Extracting Stock Price Historical Data from Yahoo! Finance using Tickers from Wikipedia
        
def extract_stock_data(stock_name, start_date):
    row_index = ticker_df.query(f"Security == '{stock_name}'").index[0]
    stock_ticker = ticker_list[row_index]
    OHLCV_df = yf.download(stock_ticker,start_date,today)
    OHLCV_df.index = pd.to_datetime(OHLCV_df.index)
    #print(f"{stock_ticker}:{stock_name}")
    #display(OHLCV_df.head(3))
    #display(OHLCV_df.tail(3))
    return(OHLCV_df)

# Feature Engineering _ additionals features for ML Analysis

def technical_indicators():
    OHLCV_df = extract_stock_data(stock_name, start_date)
    # VWAP modified to use Annual cummulative sum
    OHLCV_df['typical_price'] = (OHLCV_df['High']+OHLCV_df['Low']+OHLCV_df['Close'])/3
    OHLCV_df['vw_typical_price'] = OHLCV_df['typical_price']*OHLCV_df['Volume']
    OHLCV_df['vwap_annual'] = OHLCV_df.groupby(OHLCV_df.index.year)['vw_typical_price'].cumsum() / OHLCV_df.groupby(OHLCV_df.index.year)['Volume'].cumsum()
    # Daily Returns
    OHLCV_df['daily_returns'] = OHLCV_df['Close'].pct_change()
    # 21Day Volatility
    OHLCV_df['volatility_21'] = OHLCV_df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    OHLCV_df['sma_21'] = OHLCV_df.ta.sma(length=21)
    OHLCV_df['sma_50'] = OHLCV_df.ta.sma(length=50)
    # Exponential Moving Avg 21Days & 50Days
    OHLCV_df['ema_21'] = OHLCV_df.ta.ema(length=21)
    OHLCV_df['ema_50'] = OHLCV_df.ta.ema(length=50)
    # RSI Momemtum Indicator 14Days
    rsi_df = OHLCV_df.ta.rsi()
    # MACD Momentum Indicator
    macd_df = OHLCV_df.ta.macd()
    # Bolinger Bands %B Indicator
    bbands_df = OHLCV_df.ta.bbands()
    OHLCV_append_df = pd.concat([OHLCV_df,rsi_df,macd_df,bbands_df],axis=1)    
    return(OHLCV_append_df)


In [13]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 11months and Testing Data of 22months alternatively 
    # with latest 33months of testing data
    train_range_months = 11
    test_range_months = 22
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    model = BayesianRidge()
    target_predictions_close_df = target_test_close.copy()
    model.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model.predict(predictors_test_scaled)
    target_predictions_close_df['Bayesian_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    model.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model.predict(predictors_test_scaled)
    target_predictions_vwap_df['Bayesian_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    return(target_predictions_df)



In [19]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def linear_model_predictions():
    target_predictions_df = predict_close_vwap()
    dict_close = {}
    dict_vwap = {}
    rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'Bayesian_Close'])**2))
    mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,'Bayesian_Close'])/target_predictions_df.loc[:today,'Close']))*100
    dict_close['Bayesian'] = [rmse_close,mape_close]
    rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'Bayesian_vwap_annual'])**2))
    mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,'Bayesian_vwap_annual'])/target_predictions_df.loc[:today,'vwap_annual']))*100
    dict_vwap['Bayesian_vwap_annual'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    
    predictions_output = target_predictions_df[['Close','vwap_annual','Bayesian_Close','Bayesian_vwap_annual']]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output['Bayesian_Close'])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output['Bayesian_vwap_annual'])

    nxt_day_predictions = predictions_output[['Close','vwap_annual']]
    model_evaluation = error_measures_close
    #display(predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=500,frame_height=300))
    
    return (model_evaluation,nxt_day_predictions)


In [20]:
# Feature Engineering _ for future prediction using stock_predictions as input
# OBV & ADX dropped to limit predicted features

def feature_engineering(df):
    # Daily Returns
    df['daily_returns'] = df['Close'].pct_change()
    # 21Day Volatility
    df['volatility_21'] = df['Close'].rolling(window=21).std()
    # Simple Moving Avg 21Days & 50Days
    df['sma_21'] = df['Close'].rolling(window=21).mean()
    df['sma_50'] = df['Close'].rolling(window=50).mean()
    # Exponential Moving Avg 21Days & 50Days
    df['ema_21'] = df['Close'].ewm(span=21, adjust=False).mean()
    df['ema_50'] = df['Close'].ewm(span=50, adjust=False).mean()
    # RSI Momemtum Indicator 14Days
    rsi_df = df.ta.rsi()
    # MACD Indicator
    macd_df = df.ta.macd()
    # Bolinger Bands %B Indicator
    bbands_df = df.ta.bbands()
    # Dropping trivial Columns
    append_df = pd.concat([df,rsi_df,macd_df,bbands_df],axis=1).dropna()
    # trim_df = df[['Close','vwap_annual']]
    return(append_df)


In [21]:
# Creating instance of best fit model _ BayesianRidge()
# By fitting the historical data

def best_fit_model_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns = ['Open','High','Low','Adj Close','Volume',
                   'typical_price','vw_typical_price']).dropna()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 11
    test_range_months = 22
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model = BayesianRidge()
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model}_Close'] = model_prediction_close[-1]

    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model}_vwap_annual'] = model_prediction_vwap[-1]

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    #display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model}_vwap_annual'])

    future_predictions = target_future_df[['Close','vwap_annual']]
    return (future_predictions)


In [69]:
# Scrapping for List of stocks in S&P500 for Linear Model ML Prediction Analysis
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
  
wiki_data = pd.read_html(url)

ticker_df = wiki_data[0]
display(ticker_df.head(2))
display(ticker_df.tail(2))

# Pick Stock from the list for analysis

ticker_list= ticker_df['Symbol'].to_list()
stock_list = ticker_df['Security'].to_list()
#print(stock_list)
#stock_name = input('Enter Stock_Name from the above displayed list for Analysis:')
#start_date = input('Enter Start Date for Historical Data in yyyy-mm-dd:')


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
501,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
502,ZTS,Zoetis,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [65]:
stock_list = ['3M', 'A. O. Smith', 'Abbott', 'AbbVie', 'Accenture', 'Adobe Inc.', 'Advanced Micro Devices', 'AES Corporation', 'Aflac', 'Agilent Technologies', 'Air Products and Chemicals', 'Airbnb', 'Akamai', 'Albemarle Corporation', 'Alexandria Real Estate Equities', 'Align Technology', 'Allegion', 'Alliant Energy', 'Allstate', 'Alphabet Inc. (Class A)', 'Alphabet Inc. (Class C)', 'Altria', 'Amazon', 'Amcor', 'Ameren', 'American Airlines Group', 'American Electric Power', 'American Express', 'American International Group', 'American Tower', 'American Water Works', 'Ameriprise Financial', 'Ametek', 'Amgen', 'Amphenol', 'Analog Devices', 'Ansys', 'Aon', 'APA Corporation', 'Apple Inc.', 'Applied Materials', 'Aptiv', 'Arch Capital Group', 'Archer-Daniels-Midland', 'Arista Networks', 'Arthur J. Gallagher & Co.', 'Assurant', 'AT&T', 'Atmos Energy', 'Autodesk', 'Automatic Data Processing', 'AutoZone', 'AvalonBay Communities', 'Avery Dennison', 'Axon Enterprise', 'Baker Hughes', 'Ball Corporation', 'Bank of America', 'Bank of New York Mellon', 'Bath & Body Works, Inc.', 'Baxter International', 'Becton Dickinson', 'Best Buy', 'Bio-Rad', 'Bio-Techne', 'Biogen', 'BlackRock', 'Blackstone', 'Boeing', 'Booking Holdings', 'BorgWarner', 'Boston Properties', 'Boston Scientific', 'Bristol Myers Squibb', 'Broadcom Inc.', 'Broadridge Financial Solutions', 'Brown & Brown', 'Builders FirstSource', 'Bunge Global SA', 'Cadence Design Systems', 'Caesars Entertainment', 'Camden Property Trust', 'Campbell Soup Company', 'Capital One', 'Cardinal Health', 'CarMax', 'Carnival', 'Carrier Global', 'Catalent', 'Caterpillar Inc.', 'Cboe Global Markets', 'CBRE Group', 'CDW', 'Celanese', 'Cencora', 'Centene Corporation', 'CenterPoint Energy', 'CF Industries', 'CH Robinson', 'Charles River Laboratories', 'Charles Schwab Corporation', 'Charter Communications', 'Chevron Corporation', 'Chipotle Mexican Grill', 'Chubb Limited', 'Church & Dwight', 'Cigna', 'Cincinnati Financial', 'Cintas', 'Cisco', 'Citigroup', 'Citizens Financial Group', 'Clorox', 'CME Group', 'CMS Energy', 'Coca-Cola Company (The)', 'Cognizant', 'Colgate-Palmolive', 'Comcast', 'Comerica', 'Conagra Brands', 'ConocoPhillips', 'Consolidated Edison', 'Constellation Brands', 'Constellation Energy', 'CooperCompanies', 'Copart', 'Corning Inc.', 'Corpay', 'Corteva', 'CoStar Group', 'Costco', 'Coterra', 'Crown Castle', 'CSX', 'Cummins', 'CVS Health', 'Danaher Corporation', 'Darden Restaurants', 'DaVita Inc.', 'Dayforce', 'Deckers Brands', 'John Deere', 'Delta Air Lines', 'Dentsply Sirona', 'Devon Energy', 'Dexcom', 'Diamondback Energy', 'Digital Realty', 'Discover Financial', 'Dollar General', 'Dollar Tree', 'Dominion Energy', "Domino's", 'Dover Corporation', 'Dow Inc.', 'DR Horton', 'DTE Energy', 'Duke Energy', 'DuPont', 'Eastman Chemical Company', 'Eaton Corporation', 'eBay', 'Ecolab', 'Edison International', 'Edwards Lifesciences', 'Electronic Arts', 'Elevance Health', 'Eli Lilly and Company', 'Emerson Electric', 'Enphase', 'Entergy', 'EOG Resources', 'EPAM Systems', 'EQT', 'Equifax', 'Equinix', 'Equity Residential', 'Essex Property Trust', 'Estée Lauder Companies (The)', 'Etsy', 'Everest Re', 'Evergy', 'Eversource', 'Exelon', 'Expedia Group', 'Expeditors International', 'Extra Space Storage', 'ExxonMobil', 'F5, Inc.', 'FactSet', 'Fair Isaac', 'Fastenal', 'Federal Realty', 'FedEx', 'Fidelity National Information Services', 'Fifth Third Bank', 'First Solar', 'FirstEnergy', 'Fiserv', 'FMC Corporation', 'Ford Motor Company', 'Fortinet', 'Fortive', 'Fox Corporation (Class A)', 'Fox Corporation (Class B)', 'Franklin Templeton', 'Freeport-McMoRan', 'Garmin', 'Gartner', 'GE HealthCare', 'Gen Digital', 'Generac', 'General Dynamics', 'General Electric', 'General Mills', 'General Motors', 'Genuine Parts Company', 'Gilead Sciences', 'Global Payments', 'Globe Life', 'Goldman Sachs', 'Halliburton', 'Hartford (The)', 'Hasbro', 'HCA Healthcare', 'Healthpeak', 'Henry Schein', "Hershey's", 'Hess Corporation', 'Hewlett Packard Enterprise', 'Hilton Worldwide', 'Hologic', 'Home Depot (The)', 'Honeywell', 'Hormel Foods', 'Host Hotels & Resorts', 'Howmet Aerospace', 'HP Inc.', 'Hubbell Incorporated', 'Humana', 'Huntington Bancshares', 'Huntington Ingalls Industries', 'IBM', 'IDEX Corporation', 'Idexx Laboratories', 'Illinois Tool Works', 'Illumina', 'Incyte', 'Ingersoll Rand', 'Insulet', 'Intel', 'Intercontinental Exchange', 'International Flavors & Fragrances', 'International Paper', 'Interpublic Group of Companies (The)', 'Intuit', 'Intuitive Surgical', 'Invesco', 'Invitation Homes', 'IQVIA', 'Iron Mountain', 'J.B. Hunt', 'Jabil', 'Jack Henry & Associates', 'Jacobs Solutions', 'Johnson & Johnson', 'Johnson Controls', 'JPMorgan Chase', 'Juniper Networks', 'Kellanova', 'Kenvue', 'Keurig Dr Pepper', 'KeyCorp', 'Keysight', 'Kimberly-Clark', 'Kimco Realty', 'Kinder Morgan', 'KLA Corporation', 'Kraft Heinz', 'Kroger', 'L3Harris', 'LabCorp', 'Lam Research', 'Lamb Weston', 'Las Vegas Sands', 'Leidos', 'Lennar', 'Linde plc', 'Live Nation Entertainment', 'LKQ Corporation', 'Lockheed Martin', 'Loews Corporation', "Lowe's", 'Lululemon Athletica', 'LyondellBasell', 'M&T Bank', 'Marathon Oil', 'Marathon Petroleum', 'MarketAxess', 'Marriott International', 'Marsh McLennan', 'Martin Marietta Materials', 'Masco', 'Mastercard', 'Match Group', 'McCormick & Company', "McDonald's", 'McKesson', 'Medtronic', 'Merck & Co.', 'Meta Platforms', 'MetLife', 'Mettler Toledo', 'MGM Resorts', 'Microchip Technology', 'Micron Technology', 'Microsoft', 'Mid-America Apartment Communities', 'Moderna', 'Mohawk Industries', 'Molina Healthcare', 'Molson Coors Beverage Company', 'Mondelez International', 'Monolithic Power Systems', 'Monster Beverage', "Moody's Corporation", 'Morgan Stanley', 'Mosaic Company (The)', 'Motorola Solutions', 'MSCI', 'Nasdaq, Inc.', 'NetApp', 'Netflix', 'Newmont', 'News Corp (Class A)', 'News Corp (Class B)', 'NextEra Energy', 'Nike, Inc.', 'NiSource', 'Nordson Corporation', 'Norfolk Southern Railway', 'Northern Trust', 'Northrop Grumman',
              'Norwegian Cruise Line Holdings', 'NRG Energy', 'Nucor', 'Nvidia', 'NVR, Inc.', 'NXP Semiconductors', "O'Reilly Auto Parts", 'Occidental Petroleum', 'Old Dominion', 'Omnicom Group', 'ON Semiconductor', 'ONEOK', 'Oracle Corporation', 'Otis Worldwide', 'Paccar', 'Packaging Corporation of America', 'Palo Alto Networks', 'Paramount Global', 'Parker Hannifin', 'Paychex', 'Paycom', 'PayPal', 'Pentair', 'PepsiCo', 'Pfizer', 'PG&E Corporation', 'Philip Morris International', 'Phillips 66', 'Pinnacle West', 'Pioneer Natural Resources', 'PNC Financial Services', 'Pool Corporation', 'PPG Industries', 'PPL Corporation', 'Principal Financial Group', 'Procter & Gamble', 'Progressive Corporation', 'Prologis', 'Prudential Financial', 'Public Service Enterprise Group', 'PTC', 'Public Storage', 'PulteGroup', 'Qorvo', 'Quanta Services', 'Qualcomm', 'Quest Diagnostics', 'Ralph Lauren Corporation', 'Raymond James', 'RTX Corporation', 'Realty Income', 'Regency Centers', 'Regeneron', 'Regions Financial Corporation', 'Republic Services', 'ResMed', 'Revvity', 'Robert Half', 'Rockwell Automation', 'Rollins, Inc.', 'Roper Technologies', 'Ross Stores', 'Royal Caribbean Group', 'S&P Global', 'Salesforce', 'SBA Communications', 'Schlumberger', 'Seagate Technology', 'Sempra Energy', 'ServiceNow', 'Sherwin-Williams', 'Simon Property Group', 'Skyworks Solutions', 'J.M. Smucker Company (The)', 'Snap-on', 'Southern Company', 'Southwest Airlines', 'Stanley Black & Decker', 'Starbucks', 'State Street Corporation', 'Steel Dynamics', 'Steris', 'Stryker Corporation', 'Supermicro', 'Synchrony Financial', 'Synopsys', 'Sysco', 'T-Mobile US', 'T. Rowe Price', 'Take-Two Interactive', 'Tapestry, Inc.', 'Targa Resources', 'Target Corporation', 'TE Connectivity', 'Teledyne Technologies', 'Teleflex', 'Teradyne', 'Tesla, Inc.', 'Texas Instruments', 'Textron', 'Thermo Fisher Scientific', 'TJX Companies', 'Tractor Supply', 'Trane Technologies', 'TransDigm Group', 'Travelers Companies (The)', 'Trimble Inc.', 'Truist', 'Tyler Technologies', 'Tyson Foods', 'U.S. Bank', 'Uber', 'UDR, Inc.', 'Ulta Beauty', 'Union Pacific Corporation', 'United Airlines Holdings', 'United Parcel Service', 'United Rentals', 'UnitedHealth Group', 'Universal Health Services', 'Valero Energy', 'Ventas', 'Veralto', 'Verisign', 'Verisk', 'Verizon', 'Vertex Pharmaceuticals', 'VF Corporation', 'Viatris', 'Vici Properties', 'Visa Inc.', 'Vulcan Materials Company', 'Berkley', 'Wabtec', 'Walgreens Boots Alliance', 'Walmart', 'Walt Disney', 'Warner Bros. Discovery', 'Waste Management', 'Waters Corporation', 'WEC Energy Group', 'Wells Fargo', 'Welltower', 'West Pharmaceutical Services', 'Western Digital', 'WestRock', 'Weyerhaeuser', 'Williams Companies', 'Willis Towers Watson', 'W. W. Grainger', 'Wynn Resorts', 'Xcel Energy', 'Xylem Inc.', 'Yum! Brands', 'Zebra Technologies', 'Zimmer Biomet', 'Zoetis']

In [ ]:
# display predictions
master_df=pd.DataFrame()
model_dict={}

In [64]:

for stock in stock_list[12:]:
    stock_name = stock
    start_date = '2010-01-01'

    model,stock_predictions = linear_model_predictions()
    k=12
    model_dict['Stock']=stock_name
    model_dict['RMSE']= round(model.iloc[0,0],2)
    model_dict['MAPE']=round(model.iloc[0,1],2)
    model_df = pd.DataFrame(model_dict,index=[k])
    k=k+1
    future_stock_price = best_fit_model_future_prediction(stock_predictions)
    # Predicting Stocks on Future Dates _ Next 14 Days
    for i in list(range(1,14)):
        future_stock_price = best_fit_model_future_prediction(future_stock_price)
        
    master_df[f'{stock}'] = future_stock_price['Close']




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

ValueError: attempt to get argmax of an empty sequence

In [68]:
master_df_2=master_df
master_df_2.iloc[:,9:]

,Agilent Technologies,Air Products and Chemicals,Akamai,Albemarle Corporation,Alexandria Real Estate Equities,Align Technology,Allegion,Alliant Energy,Allstate,Alphabet Inc. (Class A),...,Blackstone,Boeing,Booking Holdings,BorgWarner,Boston Properties,Boston Scientific,Bristol Myers Squibb,Broadcom Inc.,Broadridge Financial Solutions,Brown & Brown
Date,,,,,,,,,,,,,,,,,,,,,
2014-10-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-27,38.619457,118.917671,55.290001,56.529999,82.389999,51.320000,NaN,30.295000,63.130001,27.493999,...,30.389999,122.120003,1134.260010,48.653168,122.900002,13.100000,53.529999,81.570000,42.529999,15.780000
2014-10-28,39.098713,120.000000,57.150002,57.990002,82.320000,52.209999,NaN,30.510000,63.230000,27.947001,...,30.549999,123.610001,1156.619995,50.281689,123.120003,13.210000,54.200001,84.199997,43.320000,15.945000
2014-10-29,38.984264,119.195190,56.209999,57.189999,81.870003,52.180000,NaN,30.200001,62.980000,27.922501,...,30.000000,123.080002,1144.219971,50.052818,124.739998,13.140000,54.160000,84.879997,43.419998,15.790000
2014-10-30,38.855507,123.570770,59.830002,57.349998,82.489998,51.950001,NaN,30.955000,63.910000,28.013500,...,29.760000,123.820000,1146.589966,47.860916,125.570000,13.180000,58.980000,82.389999,43.580002,15.795000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-11,147.136870,243.498593,111.674028,131.105716,126.304787,336.861938,136.961703,50.449038,172.161350,148.624321,...,131.953744,204.580606,3570.782615,34.675616,67.140998,68.499854,53.445890,1275.185987,202.409390,85.727746
2024-04-12,147.223739,243.580508,111.862558,131.411591,126.130211,337.335374,137.312288,50.514808,172.280682,148.125777,...,131.912144,205.519769,3562.816449,34.715111,67.503823,68.501811,53.475760,1270.605634,202.106231,85.625352
2024-04-15,147.307663,243.601678,112.053073,131.723549,125.970138,337.832510,137.721723,50.577899,172.376198,147.642535,...,131.877948,206.449551,3555.047503,34.661175,67.872866,68.500143,53.505397,1266.156011,201.814810,85.533663


In [61]:
master_df_1=master_df
master_df_1

,3M,A. O. Smith,Abbott,AbbVie,Accenture,Adobe Inc.,Advanced Micro Devices,AES Corporation,Aflac,Agilent Technologies,Air Products and Chemicals
Date,,,,,,,,,,,
2014-10-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-10-27,149.559998,25.690001,42.240002,NaN,79.019997,67.029999,2.660000,13.420000,29.375000,38.619457,118.917671
2014-10-28,151.059998,26.309999,42.500000,NaN,79.470001,68.370003,2.720000,13.600000,29.770000,39.098713,120.000000
2014-10-29,151.009995,26.184999,42.740002,NaN,79.550003,67.930000,2.770000,13.410000,29.950001,38.984264,119.195190
2014-10-30,152.100006,26.344999,43.490002,NaN,80.620003,68.570000,2.750000,13.950000,30.020000,38.855507,123.570770
...,...,...,...,...,...,...,...,...,...,...,...
2024-04-11,105.907966,86.367470,111.240862,185.147047,335.451359,534.400724,175.062503,16.753644,85.417003,147.136870,243.498593
2024-04-12,106.034793,86.117366,111.100149,185.594039,336.006757,536.155292,174.895820,16.727593,85.346791,147.223739,243.580508
2024-04-15,106.137514,85.861859,110.963500,186.065503,336.557258,537.450169,174.623303,16.695141,85.287805,147.307663,243.601678


In [26]:
model_df

,Stock,RMSE,MAPE


In [54]:
stock_name='Berkshire Hathaway'
start_date='2013-01-01'
raw_df=extract_stock_data('Berkshire Hathaway','2013-01-01')
raw_df.head()

[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [28]:
df1=technical_indicators()
df1.head()

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,typical_price,vw_typical_price,vwap_annual,daily_returns,...,ema_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,94.190002,94.790001,93.959999,94.779999,65.751122,3206700,94.510000,3.030652e+08,94.510000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-03,94.339996,94.930000,94.129997,94.669998,65.674812,2704600,94.576665,2.557920e+08,94.540501,-0.001161,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-04,94.790001,95.480003,94.540001,95.370003,66.160431,2704900,95.130002,2.573171e+08,94.725564,0.007394,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-07,95.019997,95.730003,94.760002,95.489998,66.243675,2745800,95.326668,2.617480e+08,94.870830,0.001258,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013-01-08,95.169998,95.750000,95.099998,95.500000,66.250618,2655500,95.449999,2.534675e+08,94.980549,0.000105,...,NaN,NaN,NaN,NaN,NaN,94.439181,95.162,95.884818,1.519132,0.733807


In [29]:
df4=predict_close_vwap()

display(df4)

[*********************100%%**********************]  1 of 1 completed


,Close,Bayesian_Close,vwap_annual,Bayesian_vwap_annual
Date,,,,
2014-02-12,130.139999,130.252816,132.107316,131.867875
2014-02-13,132.119995,130.084729,132.081423,131.758526
2014-02-14,131.800003,132.189566,132.080907,131.930883
2014-02-18,130.559998,131.770713,132.058917,131.931932
2014-02-19,131.559998,130.684827,132.041676,131.848594
...,...,...,...,...
2024-03-21,106.779999,108.066971,98.834219,99.280510
2024-03-22,104.839996,107.077610,98.959043,99.396540
2024-03-25,102.629997,105.141733,99.044819,99.511543


In [41]:
m,df5=linear_model_predictions()
m.iloc[0,0]
model_df={}
model_df['Stock']='BRK.B'
model_df['RMSE']= round(m.iloc[0,0],2)
model_df['MAPE']=round(m.iloc[0,1],2)
model = pd.DataFrame(model_df,index=[0])
model

[*********************100%%**********************]  1 of 1 completed


,Stock,RMSE,MAPE
0,3M,2.34,1.04


In [ ]:
m

In [125]:
df6=feature_engineering(df5)
df6.tail()

,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,
2024-03-22,106.779999,98.834219,-0.010105,6.579205,97.769049,97.404601,100.282690,98.597294,72.798125,3.338086,1.423727,1.914359,103.922563,106.576001,109.229439,4.979429,0.538440
2024-03-25,104.839996,98.959043,-0.018168,6.639403,98.352858,97.323000,100.696991,98.842105,64.959413,3.246122,1.065410,2.180711,103.951869,106.584000,109.216131,4.939073,0.168709
2024-03-26,102.629997,99.044819,-0.021080,6.525659,98.867620,97.213200,100.872719,98.990650,57.379631,2.960781,0.624056,2.336725,101.922956,106.045999,110.169043,7.775953,0.085743
2024-03-27,104.589996,99.108855,0.019098,6.457956,99.452858,97.152200,101.210653,99.210232,61.653213,2.859836,0.418488,2.441347,101.694411,105.341998,108.989585,6.925229,0.396918
2024-03-28,105.076519,99.798491,0.004652,6.296883,100.101263,97.132131,101.562095,99.440283,62.654246,2.786967,0.276496,2.510471,102.137539,104.783302,107.429065,5.049971,0.555413


In [126]:
df7=best_fit_model_future_prediction(df5)
df7.tail(1)

[*********************100%%**********************]  1 of 1 completed


,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,
2022-06-07,146.869995,153.820713,0.004514,25.135141,162.620475,185.5946,160.179922,176.504251,21.592546,-12.906143,0.387451,-13.293594,145.163814,146.729999,148.296184,2.134785,0.544693


,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,
2024-03-28,105.076519,99.798491,0.004652,6.296883,100.101263,97.132131,101.562095,99.440283,62.654246,2.786967,0.276496,2.510471,102.137539,104.783302,107.429065,5.049971,0.555413


,Close,vwap_annual
Date,,
2024-03-29,105.262937,100.399971


In [127]:
for i in list(range(1,25)):
    df7=best_fit_model_future_prediction(df7)
    display(df7.tail(1))

[*********************100%%**********************]  1 of 1 completed


,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,
2022-08-18,146.679993,148.53298,-0.005087,5.720709,144.425713,137.209999,144.771572,141.287619,58.34513,3.799827,-0.097741,3.897567,145.227206,149.607996,153.988785,5.856358,0.165813


,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,
2024-03-29,105.262937,100.399971,0.001774,6.114456,100.727117,97.108589,101.898535,99.668622,63.05224,2.712988,0.162013,2.550975,102.575572,104.479889,106.384207,3.645329,0.705598


,Close,vwap_annual
Date,,
2024-04-01,105.396877,100.923409


[*********************100%%**********************]  1 of 1 completed


...

,Close,vwap_annual
Date,,
2024-04-11,105.269513,101.13158


[*********************100%%**********************]  1 of 1 completed


,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,Close,...,volatility_21,sma_21,sma_50,ema_21,ema_50,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,,,,,


,Close,vwap_annual,daily_returns,volatility_21,sma_21,sma_50,ema_21,ema_50,RSI_14,Close,...,volatility_21,sma_21,sma_50,ema_21,ema_50,BBL_5_2.0,BBM_5_2.0,BBU_5_2.0,BBB_5_2.0,BBP_5_2.0
Date,,,,,,,,,,,,,,,,,,,,,


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- MACD_12_26_9
- MACDh_12_26_9
- MACDs_12_26_9


## Predicting Stock Price for future dates

In [ ]:
#Predicting Day1
future_stock_price = best_fit_model_future_prediction(stock_predictions)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price = best_fit_model_future_prediction(future_stock_price)
    
#Display Future Stock Price Predictions
display(future_stock_price.loc[today:,:])

future_plot = future_stock_price.loc[today:,'Close'].hvplot(frame_width=500,frame_height=300)

display(future_plot)
display(model_plot*future_plot)

## Predicting Using Regressor Models

In [ ]:
# Single Target Regressor Models
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

svr_rbf = SVR(kernel='rbf',C=100,gamma='auto')
# svr_poly = SVR(kernel='poly',C=100,degree=2, gamma='auto')

knn_uniform = KNeighborsRegressor(n_neighbors=2,weights='uniform')
knn_distance = KNeighborsRegressor(n_neighbors=5,weights='distance')

dtree = DecisionTreeRegressor(max_depth=10)

ens = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))

# mlp = MLPRegressor(random_state=1, max_iter=100)

In [ ]:
# Predicting Stock Close & Volume-Weighted Average Price(VWAP) using Linear Models

def predict_regressor_close_vwap():
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_test_scaled = predictors_scaled.transform(predictors_test)

    list_of_regressors = [svr_rbf,knn_uniform,knn_distance,dtree,ens]
    target_predictions_close_df = target_test_close.copy()
    for model in list_of_regressors:
        model_instance_close = model
        model_instance_close.fit(predictors_train_scaled,target_train_close)
        model_prediction_close = model_instance_close.predict(predictors_test_scaled)
        target_predictions_close_df[f'{model}_Close'] = model_prediction_close

    target_predictions_vwap_df = target_test_vwap.copy()
    for model in list_of_regressors:
        model_instance_vwap = model
        model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
        model_prediction_vwap = model_instance_vwap.predict(predictors_test_scaled)
        target_predictions_vwap_df[f'{model}_vwap_annual'] = model_prediction_vwap

    target_predictions_df = pd.concat([target_predictions_close_df,target_predictions_vwap_df],axis=1)

    display(target_predictions_df.tail())
    display(target_predictions_close_df.hvplot(frame_width=500,frame_height=300))
    return(target_predictions_close_df,target_predictions_vwap_df,target_predictions_df)


In [ ]:
# Calculating Root_Mean_Square_Error(RMSE) 
# and Mean_Absolute_Percentage_Error(MAPE)

def regressor_model_predictions():
    display(extract_stock_data(stock_name, start_date).head(3))
    display(extract_stock_data(stock_name, start_date).tail(3))
    display(technical_indicators().tail())
    today = datetime.today().strftime('%Y-%m-%d')
    target_close_predictions_df,target_vwap_predictions_df,target_predictions_df = predict_regressor_close_vwap()
    column_range_close = list(target_close_predictions_df.columns)
    column_range_vwap = list(target_vwap_predictions_df.columns)
    dict_close = {}
    dict_vwap = {}
    for i in column_range_close[1:]:
        rmse_close = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])**2))
        mape_close = np.mean(np.abs(
            (target_predictions_df.loc[:today,'Close']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'Close']))*100
        dict_close[f'{i}'] = [rmse_close,mape_close]
    for i in column_range_vwap[1:]:  
        rmse_vwap = np.sqrt(np.mean(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])**2))
        mape_vwap = np.mean(np.abs(
            (target_predictions_df.loc[:today,'vwap_annual']-target_predictions_df.loc[:today,i])/target_predictions_df.loc[:today,'vwap_annual']))*100
        dict_vwap[f'{i}'] = [rmse_vwap,mape_vwap]
    error_measures_close = pd.DataFrame.from_dict(dict_close, orient = 'index', columns = ['RMSE_close','MAPE_close'])
    error_measures_vwap = pd.DataFrame.from_dict(dict_vwap, orient = 'index', columns = ['RMSE_vwap','MAPE_vwap'])
    error_measures_close = error_measures_close.sort_values(by=['MAPE_close'])
    error_measures_vwap = error_measures_vwap.sort_values(by=['MAPE_vwap'])

    display(error_measures_close)
    display(error_measures_close.hvplot.bar(rot=90,frame_width=700,frame_height=300))

    best_estimate_close = error_measures_close.index[0]
    best_estimate_vwap = error_measures_vwap.index[0]

    predictions_output = target_predictions_df[['Close','vwap_annual',best_estimate_close, best_estimate_vwap]]
    
    nxt_day = predictions_output.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    predictions_output.loc[nxt_day] = np.nan
    predictions_output = predictions_output.shift()
    predictions_output['Close']=predictions_output['Close'].fillna(predictions_output[best_estimate_close])
    predictions_output['vwap_annual']=predictions_output['vwap_annual'].fillna(predictions_output[best_estimate_vwap])

    predictions = predictions_output[['Close','vwap_annual']]
    best_model = best_estimate_close.removesuffix('_Close')
    comparison_plot = predictions_output[['Close',best_estimate_close]].loc['2024':today].hvplot(frame_width=300,frame_height=300)
    
    return (comparison_plot,best_model,predictions)



In [ ]:
regressor_plot,regressor,stock_predictions_regr = regressor_model_predictions()

display(stock_predictions_regr.tail())
print(f'Best fit Regressor Model: {regressor}')
regressor_plot


In [ ]:
# Creating instance of best fit regressor model
# By fitting the historical data

def best_fit_regr_future_prediction(df):
    # Next Day's "Close" as target
    predictors_df = technical_indicators()
    predictors_df = predictors_df.drop(columns=['Volume','OBV_in_million','adx_14'])
    target_close_df = predictors_df[['Close']].shift(-1)
    target_vwap_df = predictors_df[['vwap_annual']].shift(-1)

    predictors_future_df = feature_engineering(df)
    target_future_close_df = predictors_future_df[['Close']].shift(-1)
    target_future_vwap_df = predictors_future_df[['vwap_annual']].shift(-1)

    # Training Data of 27months and Testing Data of 6months alternatively 
    # with latest 33months of testing data
    train_range_months = 27
    test_range_months = 6
    split_range = list(range(len(predictors_df)//21//(train_range_months+test_range_months)))
    predictors_train = pd.DataFrame()
    
    for i in split_range:
        train_df = predictors_df.iloc[21*33*i:(21*33*i)+(21*(33-test_range_months)),:]
        predictors_train = pd.concat([predictors_train,train_df],axis=0)
        
    predictors_test = predictors_df.drop(index = predictors_train.index)
    target_train_close = target_close_df.drop(index = predictors_test.index)
    target_test_close = target_close_df.drop(index = predictors_train.index)
    target_train_vwap = target_vwap_df.drop(index = predictors_test.index)
    target_test_vwap = target_vwap_df.drop(index = predictors_train.index)
    
    #Using StandardScaler to fit predictors train data and transform both train and test data 
    predictors_scaled = scaler.fit(predictors_train)
    predictors_train_scaled = predictors_scaled.transform(predictors_train)
    predictors_future_scaled = predictors_scaled.transform(predictors_future_df)

    best_model_close = ens
    target_future_close = target_future_close_df.copy()
    model_instance_close = best_model_close
    model_instance_close.fit(predictors_train_scaled,target_train_close)
    model_prediction_close = model_instance_close.predict(predictors_future_scaled)
    target_future_close[f'{best_model_close}_Close'] = model_prediction_close

    best_model_vwap = ens
    target_future_vwap = target_future_vwap_df.copy()
    model_instance_vwap = best_model_vwap
    model_instance_vwap.fit(predictors_train_scaled,target_train_vwap)
    model_prediction_vwap = model_instance_vwap.predict(predictors_future_scaled)
    target_future_vwap[f'{best_model_vwap}_vwap_annual'] = model_prediction_vwap

    target_future_df = pd.concat([target_future_close,target_future_vwap],axis=1)

    display(target_future_df.tail(1))

    nxt_day = target_future_df.index[-1]+pd.Timedelta(days=1)
    #Checking for Federal Holidays
    cal = USFederalHolidayCalendar()
    holidays = pd.to_datetime(cal.holidays(today, '2024-12-31'))
    hol = list(holidays)
    for i in hol:
        if nxt_day==i:
            nxt_day = nxt_day+pd.Timedelta(days=1)
    #Checking for Weekends
    if nxt_day.weekday()==5:
        nxt_day = nxt_day+pd.Timedelta(days=2)
    elif nxt_day.weekday()==6:
        nxt_day = nxt_day+pd.Timedelta(days=1)
    else:
        nxt_day = nxt_day
    
    target_future_df.loc[nxt_day] = np.nan
    target_future_df = target_future_df.shift()
    target_future_df['Close']=target_future_df['Close'].fillna(target_future_df[f'{best_model_close}_Close'])
    target_future_df['vwap_annual']=target_future_df['vwap_annual'].fillna(target_future_df[f'{best_model_vwap}_vwap_annual'])

    predictions = target_future_df[['Close','vwap_annual']]
    
    return (predictions)


In [ ]:
#Predicting Day1
future_stock_price_regr = best_fit_regr_future_prediction(stock_predictions_regr)
# Predicting Stocks on Future Dates _ Next 51 Days
for i in list(range(1,14)):
    future_stock_price_regr = best_fit_regr_future_prediction(future_stock_price_regr)

#Display Regressor Model Predictions
display(future_stock_price_regr.loc[today:,:])

future_plot_regr = future_stock_price_regr.loc[today:,'Close'].hvplot(frame_width=300,frame_height=300)

display(future_plot_regr)
display(regressor_plot*future_plot_regr)

In [ ]:
regressor_plot*future_plot*future_plot_regr

In [ ]:
linear_close = future_stock_price[['Close']].rename(columns={'Close':'Close_Linear'})
regr_close = future_stock_price_regr[['Close']].rename(columns={'Close':'Close_Regr'})

combined_df = pd.concat([linear_close,regr_close],axis=1).dropna()

display(combined_df.loc[today:])

combined_df.loc[today:].hvplot()


In [ ]:

for i in list(range(1,8)):
    future_stock_price_regr=best_fit_regr_future_prediction(future_stock_price_regr)
    future_stock_price_regr



In [ ]:
future_stock_price_regr = future_stock_price_regr[['Close','vwap_annual']].dropna()

future_stock_price_regr